In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from tqdm import tqdm
import numpy as np

from pangres import upsert
from sqlalchemy import text, create_engine

import json

load_dotenv()

True

In [ ]:
db_username=os.environ.get("db_username")
db_password=os.environ.get("db_password")
db_host=os.environ.get("db_host")
db_port=os.environ.get("db_port")
# db_name=os.environ.get("db_name")
db_name = 'lol_analytics'


def create_db_connection_string(db_username: str, db_password: str, db_host: str, db_port: int, db_name: str):
    connection_url = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
    return connection_url

In [3]:
def get_match_timeline(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}'
    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [4]:
def retrieve_match_data(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return response.json()

## Exploring data

In [7]:
def process_timeline_data(match_timeline):

    participants = match_timeline['info']['participants']
    simplified_participants = {}
    for participant in participants:
        simplified_participants[participant['participantId']] = participant['puuid']

    participants_timeline = []
    match_metadata = match_timeline['metadata']
    frames = match_timeline['info']['frames']
    match_info = match_timeline['info']

    for frame in frames:
        for participant_id, participant_dto in frame['participantFrames'].items():
            participant_timeline = {}
            participant_timeline['data_version'] = match_metadata['dataVersion']
            participant_timeline['match_id'] = match_metadata['matchId']
            participant_timeline['frame_interval'] = match_info['frameInterval']/1000
            participant_timeline['end_of_game_result'] = match_info['endOfGameResult']
            # convert into minutes
            participant_timeline['timestamp'] = round(frame['timestamp']/1000/60, 4)
            participant_timeline['participant_id'] = int(participant_id)
            participant_timeline['puuid'] = simplified_participants[int(participant_id)]
            participant_timeline['current_gold'] = participant_dto['currentGold']
            participant_timeline['level'] = participant_dto['level']
            participant_timeline['jungle_minions_killed'] = participant_dto['jungleMinionsKilled']
            participant_timeline['minions_killed'] = participant_dto['minionsKilled']
            participant_timeline['total_gold'] = participant_dto['totalGold']
            participant_timeline['xp'] = participant_dto['xp']

            champion_stats = participant_dto['championStats']
            participant_timeline['health'] = champion_stats['health']
            participant_timeline['health_max'] = champion_stats['healthMax']
            participant_timeline['mana'] = champion_stats['power']
            participant_timeline['mana_max'] = champion_stats['powerMax']
            

            player_position = participant_dto['position']
            participant_timeline['x'] = player_position['x']
            participant_timeline['y'] = player_position['y']

            participants_timeline.append(participant_timeline)

    participants_timeline_df = pd.DataFrame(participants_timeline)
    return participants_timeline_df

In [13]:
connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)
db_engine = create_engine(connection_url, pool_recycle=3600)
connection = db_engine.connect()

In [27]:
with db_engine.connect() as connection:
    df = pd.read_sql(text("SELECT * FROM soloq.player_matches"), connection)
df = df[df['username'] == 'mazsu']
df.reset_index(drop=True, inplace=True)
match_ids = df['match_id'].to_list()

api_key = os.getenv("riot_api_key")


dataframe_list = []
for match_id in tqdm(match_ids):
    match_timeline = get_match_timeline(match_id=match_id, api_key=api_key)
    participants_timeline_df = process_timeline_data(match_timeline)
    dataframe_list.append(participants_timeline_df)



100%|██████████| 97/97 [00:43<00:00,  2.22it/s]


In [28]:
all_matches_timeline_dataframe = pd.concat(dataframe_list)

all_matches_timeline_dataframe['uuid'] = all_matches_timeline_dataframe["match_id"] + '_' + all_matches_timeline_dataframe["participant_id"].astype(str) + '_' + all_matches_timeline_dataframe["timestamp"].astype(str)
print(all_matches_timeline_dataframe.shape[0])
all_matches_timeline_dataframe = all_matches_timeline_dataframe.set_index('uuid')

30510


In [29]:
connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)
db_engine = create_engine(connection_url, pool_recycle=3600)
connection = db_engine.connect()
upsert(con=connection, df=all_matches_timeline_dataframe, schema='soloq', table_name='player_matches_timeline', create_schema=True, if_row_exists='update')

In [30]:
connection.commit()